# StudyBuddy Prototyping
copied from Introduction to LangChain v0.1.0 and LCEL: LangChain Powered RAG


#### TaInstalling Required Libraries


In [ ]:
!pip install -qU langchain langchain-core langchain-community langchain-openai

Now we can get our Qdrant dependencies!

In [ ]:
!pip install -qU qdrant-client

Let's finally get `tiktoken` and `pymupdf` so we can leverage them later on!

In [ ]:
!pip install -qU tiktoken pymupdf

#### Set Environment 

We will read from .env
and use config to control the constants

In [1]:
path = 'studybuddy_utils/'

In [2]:
from dotenv import load_dotenv
load_dotenv()

# tbd make sure, the key has been loaded

True

In [1]:
from studybuddy_utils.config import Config

Set up LangSmith

In [2]:
from uuid import uuid4
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"StudyBuddy - Development - {uuid4().hex[0:8]}"


#### Read and chunk docs

In [3]:
from studybuddy_utils.text_utils import SBDocLoader
sbdocs_loader = SBDocLoader(path+Config.pdf_path)

chunks = sbdocs_loader.load_and_parse_pdf_dir()

NameError: name 'path' is not defined

### Create embeddings and setup vector store

In [7]:
from studybuddy_utils.vectorstore import IndexBuilder
index = IndexBuilder(chunks)
qdrant_retriever = index.retriever

EmbeddingModel set text-embedding-ada-002


### Formulate the question

In [9]:
question = "What is the meaning of the number 42 in Douglas Adam's Hitchhicker's Guide?"

### Execute the chain

In [10]:
from studybuddy_utils.reasoning import SimpleChain
chain = SimpleChain(qdrant_retriever)
response = chain.reason(question)

ChatModel set gpt-3.5-turbo


### Show the result

In [11]:
response

"The meaning of the number 42 in Douglas Adam's Hitchhicker's Guide is not explicitly mentioned in the provided context. Therefore, the answer is: 'I don't know the answer to that question based on the provided context.'."

# Experiment with a big dataset

In [4]:
from studybuddy_utils.config import Config
path = 'studybuddy_utils/'
pdf_path = 'data/AI-Powered_Search_v20.pdf'
pdf_path ='data/sudel.pdf'
pdf_path = 'data/AI-Powered_Search_v20.pdf'
session_uuid='dev'

In [5]:
from studybuddy_utils.text_utils import SBDocLoader
sbdocs_loader = SBDocLoader('')
chunks = sbdocs_loader.load_and_parse_pdf(pdf_path)


# of chunks: 1523
# max chunk len: 199


In [6]:
async def load_and_parse(path: str):
    documents =  await parser.aload_data(path)
    return documents

Try LlamaIndex instead

In [7]:
import nest_asyncio

nest_asyncio.apply()

from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",  # "markdown" and "text" are available
    verbose=True,
)

# sync
# documents = parser.load_data("./my_file.pdf")

# sync batch
# documents = parser.load_data(["./my_file1.pdf", "./my_file2.pdf"])

# async
documents = load_and_parse(pdf_path)

# async batch
# documents = await parser.aload_data(["./my_file1.pdf", "./my_file2.pdf"])#

In [8]:
documents

<coroutine object load_and_parse at 0x174139700>